In [2]:
import requests 
import shutil
import os
import time
import pandas as pd
df_needed = pd.read_csv('csv/ESGF_SIm-1c.csv')

/usr/local/python/anaconda3/envs/pangeo-Oct2019/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
node_pref = {'esgf-data1.llnl.gov':0,'esgf-data2.llnl.gov':0,'aims3.llnl.gov':0,'esgdata.gfdl.noaa.gov':3,'esgf-data.ucar.edu':4,
 'dpesgf03.nccs.nasa.gov':5,'crd-esgf-drc.ec.gc.ca':6, 'cmip.bcc.cma.cn':10, 'cmip.dess.tsinghua.edu.cn':10,
 'cmip.fio.org.cn':10, 'dist.nmlab.snu.ac.kr':10, 'esg-cccr.tropmet.res.in':10, 'esg-dn1.nsc.liu.se':10,
 'esg-dn2.nsc.liu.se':10, 'esg.camscma.cn':10, 'esg.lasg.ac.cn':10, 'esg1.umr-cnrm.fr':10, 'esgf-cnr.hpc.cineca.it':10,
 'esgf-data2.diasjp.net':10, 'esgf-data3.ceda.ac.uk':10, 'esgf-data3.diasjp.net':10, 'esgf-nimscmip6.apcc21.org':10, 'esgf-node2.cmcc.it':10, 
 'esgf.bsc.es':10, 'esgf.dwd.de':10, 'esgf.ichec.ie':10, 'esgf.nci.org.au':10, 'esgf.rcec.sinica.edu.tw':10, 'esgf3.dkrz.de':10, 
 'noresg.nird.sigma2.no':10, 'polaris.pknu.ac.kr':10, 'vesg.ipsl.upmc.fr':10}

nodes = df_needed.data_node.unique()

for node in nodes:
    df = df_needed[df_needed.data_node == node]
    if node == 'esg.lasg.ac.cn':
        print(node, 'too slow to measure properly')
        continue
    df = df.sort_values(by=['file_size'])
    url = df.url.values[-1]  # try on largest file
    expected_size = df[df.url==url].file_size.values[0]
    print('\n',node, 'size of file:', expected_size/1e6,'Mb')
    tic = time.perf_counter()
    try:
        r = requests.get(url, timeout=3.1, stream=True)
    except:
        print('request timed out')
        continue
    if 'netcdf' in r.headers['content-type']:
        with open('test.nc', 'wb') as f:
            shutil.copyfileobj(r.raw, f)
        toc = time.perf_counter()
        actual_size = os.path.getsize('test.nc')
        if int(expected_size) == actual_size:
            print(expected_size, actual_size)
            speed = actual_size/(toc-tic)/1.0e6
            #print(f"size:{actual_size} in seconds:{toc-tic}")
            print(f"Download speed from {node} was {speed:0.1f} Mb/s")
        else:
            print('download incomplete')
    else:
        print('did not return a netcdf file')


 aims3.llnl.gov size of file: 2406.245744 Mb
2406245744 2406245744
Download speed from aims3.llnl.gov was 17.6 Mb/s

 esgf-data1.llnl.gov size of file: 2406.245936 Mb
2406245936 2406245936
Download speed from esgf-data1.llnl.gov was 17.5 Mb/s

 esgf-nimscmip6.apcc21.org size of file: 576.075364 Mb
request timed out

 esgf-data2.diasjp.net size of file: 683.968848 Mb
request timed out

 dpesgf03.nccs.nasa.gov size of file: 62.240944 Mb
62240944 62240944
Download speed from dpesgf03.nccs.nasa.gov was 34.9 Mb/s

 esgf-data3.diasjp.net size of file: 2980.938239 Mb
request timed out

 crd-esgf-drc.ec.gc.ca size of file: 167.334206 Mb
167334206 167334206
Download speed from crd-esgf-drc.ec.gc.ca was 32.7 Mb/s

 esgf-data2.llnl.gov size of file: 49.93037 Mb
49930370 49930370
Download speed from esgf-data2.llnl.gov was 25.7 Mb/s

 esgf-data.ucar.edu size of file: 66.230409 Mb
66230409 66230409
Download speed from esgf-data.ucar.edu was 46.0 Mb/s


In [7]:
nodes

array(['aims3.llnl.gov', 'esgf-data1.llnl.gov',
       'esgf-nimscmip6.apcc21.org', 'esgf-data2.diasjp.net',
       'dpesgf03.nccs.nasa.gov', 'esg.lasg.ac.cn',
       'esgf-data3.diasjp.net', 'crd-esgf-drc.ec.gc.ca',
       'esgf-data2.llnl.gov', 'esgf-data.ucar.edu'], dtype=object)